In [1]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers

import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from numba import jit, float64, int64

In [2]:
def make_train_test(data, test_size=0.3):
    sbd = data[data['sbj'] == 0]
    X_train, X_test, y1, y2 = train_test_split(sbd, sbd, test_size=test_size, random_state=1111)
    for i in range(1, data.sbj.max()+1):
        sbd = data[data['sbj'] == i]
        tr, ts, y1, y2 = train_test_split(sbd, sbd, test_size=test_size, random_state=1111)
        X_train = pd.concat([X_train, tr])
        X_test = pd.concat([X_test, ts])
        
    return X_train.reset_index(drop=True), X_test.reset_index(drop=True)

In [3]:
@jit(nopython=True)
def Race(v, b, ndt=0, dt=0.001):
    x = np.array([0.0, 0.0])
    rt = 0
    ch = -1
    stop = False
    while not stop:
        x[0] += v[0]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        x[1] += v[1]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        rt += dt
        if x[0]>= b and x[1]<b:
            ch = 0.0
            stop = True
        elif x[1]>= b and x[0]<b:
            ch = 1.0
            stop = True
        elif x[0]>= b and x[0]>=b:
            rt = 0
            x = np.array([0.0, 0.0])
    return rt+ndt, ch

In [4]:
data_df = pd.read_csv('../../_00_data/Yang2024_prep.csv', index_col=0)

min_rt = data_df.groupby(by='sbj')['rt'].min()

tr_df, ts_df = make_train_test(data_df, test_size=0.4)

obs_train = np.asarray(tr_df[['rt', 'Right', 'sbj']])
X_train = np.asarray(tr_df[["o1_a1_val", "o1_a2_val", "o2_a1_val", "o2_a2_val",
                            "o1_a1_gaze", "o1_a2_gaze", "o2_a1_gaze", "o2_a2_gaze"]])

obs_test = np.asarray(ts_df[['rt', 'Right', 'sbj']])
X_test = np.asarray(ts_df[["o1_a1_val", "o1_a2_val", "o2_a1_val", "o2_a2_val",
                           "o1_a1_gaze", "o1_a2_gaze", "o2_a1_gaze", "o2_a2_gaze"]])

print(X_train.shape, X_test.shape)

(5830, 8) (3901, 8)


In [5]:
def fpt(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(b/(2*np.pi*t**3)**(1/2) * keras.backend.exp(-1/(2*t) * (v*t-b)**2)) + tf.cast(tf.math.less_equal(t, 0), tf.float32)*0.6**11

def phi(x):
    return 0.5*tf.math.erf(x/tf.sqrt(2.0)) + 0.5

def cdf(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(phi((v*t-b)/tf.sqrt(t)) + tf.exp(2*v*b)*phi((-v*t-b)/tf.sqrt(t))) + (tf.cast(tf.math.less_equal(t, 0), tf.float32)*0)

def G2(data, pred):

    sbj_indices = tf.cast(data[:, 2], tf.int32)

    # Gather values directly using TensorFlow operations
    gathered_min_rt = tf.gather(min_rt, sbj_indices)
    sigmoid_weights = tf.gather(model.layers[-1].weights[-1], sbj_indices)
    ndt = gathered_min_rt * tf.math.sigmoid(sigmoid_weights)

    thr_weights = tf.gather(model.layers[-1].weights[-2], sbj_indices)
    thr = tf.math.log(1 + tf.exp(thr_weights))

    scl_weights = tf.gather(model.layers[-1].weights[-3], sbj_indices)
    scl = tf.math.log(1 + tf.exp(scl_weights))

    ll1 = (1 - data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 0]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 1]))
    ll2 = (data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 1]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 0]))

    return -2*tf.reduce_sum(tf.math.log(tf.maximum(ll1 + ll2, 3e-18)), axis=-1)

In [6]:
class Out_layer(keras.layers.Layer):
    def __init__(self, units=2, input_dim=32, n_sbj_prms=10):
        super().__init__()
        self.w = self.add_weight(shape=(input_dim, units), initializer="random_normal", trainable=True)
        self.b = self.add_weight(shape=(units,), initializer="uniform", trainable=True)
        
        self.scaling = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.threshold = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.non_decision_time = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)

    def call(self, inputs):
        return tf.math.log(1 + tf.exp(tf.matmul(inputs, self.w) + self.b))
    
def make_model(layers_neuron, input_shape, output_shape, activation, n_sbj_prms):
    keras.backend.clear_session()

    model = keras.Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    
    for l in layers_neuron:
        model.add(layers.Dense(l, activation=activation))
    
    model.add(Out_layer(units=output_shape, input_dim=layers_neuron[-1], n_sbj_prms=n_sbj_prms))

    print(model.summary())
    
    return model

In [7]:
layers_neuron = [10, 10, 8, 5]
input_shape = 8
output_shape = 2
activation = 'tanh'
n_sbj = data_df.sbj.nunique()

model = make_model(layers_neuron, input_shape, output_shape, activation, n_sbj)

model.build()
model.compile(optimizer="RMSprop", loss=G2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                90        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 8)                 88        
                                                                 
 dense_3 (Dense)             (None, 5)                 45        
                                                                 
 out_layer (Out_layer)       (None, 2)                 96        
                                                                 
Total params: 429 (1.68 KB)
Trainable params: 429 (1.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [8]:
sbj_indices = tf.cast(obs_train[:, 2], tf.int32)
obs_train = tf.cast(obs_train, tf.float32)
X_train = tf.convert_to_tensor(X_train)
X_train = tf.cast(X_train, tf.float32)

min_rt = tf.convert_to_tensor(min_rt)
min_rt = tf.cast(min_rt, tf.float32)

In [9]:
history = model.fit(X_train, 
                    obs_train, 
                    shuffle=False, 
                    batch_size=X_train.shape[0], 
                    epochs=3500)

Epoch 1/3500
1/1 [==============================] - 1s 651ms/step - loss: 27603.1387
Epoch 2/3500
1/1 [==============================] - 0s 4ms/step - loss: 27487.7422
Epoch 3/3500
1/1 [==============================] - 0s 4ms/step - loss: 27404.6445
Epoch 4/3500
1/1 [==============================] - 0s 5ms/step - loss: 27334.7109
Epoch 5/3500
1/1 [==============================] - 0s 4ms/step - loss: 27272.2773
Epoch 6/3500
1/1 [==============================] - 0s 4ms/step - loss: 27214.8711
Epoch 7/3500
1/1 [==============================] - 0s 5ms/step - loss: 27161.1465
Epoch 8/3500
1/1 [==============================] - 0s 6ms/step - loss: 27110.2793
Epoch 9/3500
1/1 [==============================] - 0s 4ms/step - loss: 27061.7031
Epoch 10/3500
1/1 [==============================] - 0s 4ms/step - loss: 27015.0332
Epoch 11/3500
1/1 [==============================] - 0s 3ms/step - loss: 26969.9668
Epoch 12/3500
1/1 [==============================] - 0s 4ms/step - loss: 26926.2930

1/1 [==============================] - 0s 3ms/step - loss: 24319.8457
Epoch 99/3500
1/1 [==============================] - 0s 3ms/step - loss: 24297.5801
Epoch 100/3500
1/1 [==============================] - 0s 3ms/step - loss: 24275.4668
Epoch 101/3500
1/1 [==============================] - 0s 3ms/step - loss: 24253.4863
Epoch 102/3500
1/1 [==============================] - 0s 3ms/step - loss: 24231.6543
Epoch 103/3500
1/1 [==============================] - 0s 3ms/step - loss: 24209.9648
Epoch 104/3500
1/1 [==============================] - 0s 3ms/step - loss: 24188.4121
Epoch 105/3500
1/1 [==============================] - 0s 3ms/step - loss: 24167.0020
Epoch 106/3500
1/1 [==============================] - 0s 3ms/step - loss: 24145.7266
Epoch 107/3500
1/1 [==============================] - 0s 3ms/step - loss: 24124.5859
Epoch 108/3500
1/1 [==============================] - 0s 3ms/step - loss: 24103.5820
Epoch 109/3500
1/1 [==============================] - 0s 3ms/step - loss: 24082.7

1/1 [==============================] - 0s 3ms/step - loss: 22674.7168
Epoch 195/3500
1/1 [==============================] - 0s 3ms/step - loss: 22661.3398
Epoch 196/3500
1/1 [==============================] - 0s 3ms/step - loss: 22648.0195
Epoch 197/3500
1/1 [==============================] - 0s 3ms/step - loss: 22634.7461
Epoch 198/3500
1/1 [==============================] - 0s 3ms/step - loss: 22621.5312
Epoch 199/3500
1/1 [==============================] - 0s 3ms/step - loss: 22608.3633
Epoch 200/3500
1/1 [==============================] - 0s 3ms/step - loss: 22595.2461
Epoch 201/3500
1/1 [==============================] - 0s 3ms/step - loss: 22582.1836
Epoch 202/3500
1/1 [==============================] - 0s 3ms/step - loss: 22569.1719
Epoch 203/3500
1/1 [==============================] - 0s 3ms/step - loss: 22556.2109
Epoch 204/3500
1/1 [==============================] - 0s 3ms/step - loss: 22543.2949
Epoch 205/3500
1/1 [==============================] - 0s 3ms/step - loss: 22530.

1/1 [==============================] - 0s 4ms/step - loss: 21339.5117
Epoch 291/3500
1/1 [==============================] - 0s 4ms/step - loss: 21322.5742
Epoch 292/3500
1/1 [==============================] - 0s 4ms/step - loss: 21305.5039
Epoch 293/3500
1/1 [==============================] - 0s 4ms/step - loss: 21288.3008
Epoch 294/3500
1/1 [==============================] - 0s 4ms/step - loss: 21270.9590
Epoch 295/3500
1/1 [==============================] - 0s 4ms/step - loss: 21253.4805
Epoch 296/3500
1/1 [==============================] - 0s 5ms/step - loss: 21235.8633
Epoch 297/3500
1/1 [==============================] - 0s 4ms/step - loss: 21218.1016
Epoch 298/3500
1/1 [==============================] - 0s 4ms/step - loss: 21200.2051
Epoch 299/3500
1/1 [==============================] - 0s 5ms/step - loss: 21182.1641
Epoch 300/3500
1/1 [==============================] - 0s 4ms/step - loss: 21163.9766
Epoch 301/3500
1/1 [==============================] - 0s 4ms/step - loss: 21145.

1/1 [==============================] - 0s 4ms/step - loss: 19352.6133
Epoch 387/3500
1/1 [==============================] - 0s 4ms/step - loss: 19333.7617
Epoch 388/3500
1/1 [==============================] - 0s 5ms/step - loss: 19314.9531
Epoch 389/3500
1/1 [==============================] - 0s 4ms/step - loss: 19296.1992
Epoch 390/3500
1/1 [==============================] - 0s 4ms/step - loss: 19277.4883
Epoch 391/3500
1/1 [==============================] - 0s 4ms/step - loss: 19258.8281
Epoch 392/3500
1/1 [==============================] - 0s 4ms/step - loss: 19240.1992
Epoch 393/3500
1/1 [==============================] - 0s 4ms/step - loss: 19221.6230
Epoch 394/3500
1/1 [==============================] - 0s 4ms/step - loss: 19203.0742
Epoch 395/3500
1/1 [==============================] - 0s 4ms/step - loss: 19184.5781
Epoch 396/3500
1/1 [==============================] - 0s 4ms/step - loss: 19166.1016
Epoch 397/3500
1/1 [==============================] - 0s 4ms/step - loss: 19147.

...

1/1 [==============================] - 0s 3ms/step - loss: 9417.2969
Epoch 3065/3500
1/1 [==============================] - 0s 3ms/step - loss: 9420.1992
Epoch 3066/3500
1/1 [==============================] - 0s 3ms/step - loss: 9460.4648
Epoch 3067/3500
1/1 [==============================] - 0s 3ms/step - loss: 9502.2451
Epoch 3068/3500
1/1 [==============================] - 0s 3ms/step - loss: 9589.1641
Epoch 3069/3500
1/1 [==============================] - 0s 3ms/step - loss: 9704.0430
Epoch 3070/3500
1/1 [==============================] - 0s 3ms/step - loss: 9798.3184
Epoch 3071/3500
1/1 [==============================] - 0s 3ms/step - loss: 9662.0693
Epoch 3072/3500
1/1 [==============================] - 0s 3ms/step - loss: 9613.8887
Epoch 3073/3500
1/1 [==============================] - 0s 3ms/step - loss: 9683.9219
Epoch 3074/3500
1/1 [==============================] - 0s 3ms/step - loss: 9563.2734
Epoch 3075/3500
1/1 [==============================] - 0s 3ms/step - loss: 9486.3

1/1 [==============================] - 0s 3ms/step - loss: 9488.3828
Epoch 3161/3500
1/1 [==============================] - 0s 3ms/step - loss: 9473.6807
Epoch 3162/3500
1/1 [==============================] - 0s 3ms/step - loss: 9442.4492
Epoch 3163/3500
1/1 [==============================] - 0s 3ms/step - loss: 9451.4580
Epoch 3164/3500
1/1 [==============================] - 0s 3ms/step - loss: 9458.1084
Epoch 3165/3500
1/1 [==============================] - 0s 3ms/step - loss: 9499.6445
Epoch 3166/3500
1/1 [==============================] - 0s 3ms/step - loss: 9621.8359
Epoch 3167/3500
1/1 [==============================] - 0s 3ms/step - loss: 9579.5410
Epoch 3168/3500
1/1 [==============================] - 0s 3ms/step - loss: 9679.4561
Epoch 3169/3500
1/1 [==============================] - 0s 3ms/step - loss: 9587.2734
Epoch 3170/3500
1/1 [==============================] - 0s 3ms/step - loss: 9572.5947
Epoch 3171/3500
1/1 [==============================] - 0s 3ms/step - loss: 9528.3

1/1 [==============================] - 0s 3ms/step - loss: 9555.6777
Epoch 3257/3500
1/1 [==============================] - 0s 3ms/step - loss: 9532.7246
Epoch 3258/3500
1/1 [==============================] - 0s 3ms/step - loss: 9578.7559
Epoch 3259/3500
1/1 [==============================] - 0s 3ms/step - loss: 9487.0488
Epoch 3260/3500
1/1 [==============================] - 0s 3ms/step - loss: 9489.6143
Epoch 3261/3500
1/1 [==============================] - 0s 3ms/step - loss: 9480.4619
Epoch 3262/3500
1/1 [==============================] - 0s 3ms/step - loss: 9497.5811
Epoch 3263/3500
1/1 [==============================] - 0s 3ms/step - loss: 9542.0781
Epoch 3264/3500
1/1 [==============================] - 0s 3ms/step - loss: 9630.6826
Epoch 3265/3500
1/1 [==============================] - 0s 3ms/step - loss: 9477.6729
Epoch 3266/3500
1/1 [==============================] - 0s 3ms/step - loss: 9561.4580
Epoch 3267/3500
1/1 [==============================] - 0s 3ms/step - loss: 9487.0

1/1 [==============================] - 0s 3ms/step - loss: 9577.5107
Epoch 3353/3500
1/1 [==============================] - 0s 3ms/step - loss: 9441.5479
Epoch 3354/3500
1/1 [==============================] - 0s 4ms/step - loss: 9483.9375
Epoch 3355/3500
1/1 [==============================] - 0s 3ms/step - loss: 9508.1465
Epoch 3356/3500
1/1 [==============================] - 0s 3ms/step - loss: 9541.7842
Epoch 3357/3500
1/1 [==============================] - 0s 3ms/step - loss: 9521.4307
Epoch 3358/3500
1/1 [==============================] - 0s 3ms/step - loss: 9659.5234
Epoch 3359/3500
1/1 [==============================] - 0s 3ms/step - loss: 9504.9160
Epoch 3360/3500
1/1 [==============================] - 0s 3ms/step - loss: 9494.1846
Epoch 3361/3500
1/1 [==============================] - 0s 3ms/step - loss: 9462.8008
Epoch 3362/3500
1/1 [==============================] - 0s 3ms/step - loss: 9519.9785
Epoch 3363/3500
1/1 [==============================] - 0s 3ms/step - loss: 9441.2

1/1 [==============================] - 0s 3ms/step - loss: 9584.5127
Epoch 3449/3500
1/1 [==============================] - 0s 3ms/step - loss: 9809.5938
Epoch 3450/3500
1/1 [==============================] - 0s 3ms/step - loss: 9503.4648
Epoch 3451/3500
1/1 [==============================] - 0s 3ms/step - loss: 9395.6338
Epoch 3452/3500
1/1 [==============================] - 0s 3ms/step - loss: 9439.2441
Epoch 3453/3500
1/1 [==============================] - 0s 3ms/step - loss: 9413.1865
Epoch 3454/3500
1/1 [==============================] - 0s 3ms/step - loss: 9345.0068
Epoch 3455/3500
1/1 [==============================] - 0s 3ms/step - loss: 9342.4316
Epoch 3456/3500
1/1 [==============================] - 0s 3ms/step - loss: 9433.7637
Epoch 3457/3500
1/1 [==============================] - 0s 3ms/step - loss: 9380.0752
Epoch 3458/3500
1/1 [==============================] - 0s 3ms/step - loss: 9505.6289
Epoch 3459/3500
1/1 [==============================] - 0s 3ms/step - loss: 9485.8

In [10]:
pred_v = model.predict(X_test)
threshold = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-2])))
ndt = np.asanyarray(np.asanyarray(min_rt)*tf.math.sigmoid(model.layers[-1].weights[-1]))

122/122 [==============================] - 0s 351us/step


In [11]:
pred_bhv = {'sbj': [],
            'rt': [],
            'ch': [],
            'value_diff': [],
            'gaze_diff': []}

scaling = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-3])))
scaling = np.asanyarray([scaling[i-1] for i in ts_df['sbj']])

for n in range(20):
    for s in range(ts_df.shape[0]):
        rt, ch = Race(scaling[s]*pred_v[s,:], threshold[ts_df.sbj[s]-1], ndt[ts_df.sbj[s]-1])
        pred_bhv['sbj'].append(ts_df.sbj[s])
        pred_bhv['rt'].append(rt)
        pred_bhv['ch'].append(ch)
        pred_bhv['value_diff'].append(ts_df['Right Value - Left Value'][s])
        pred_bhv['gaze_diff'].append(ts_df['Dwell Time Advantage (R - L)'][s])
        
pred_bhv = pd.DataFrame(pred_bhv)

In [12]:
pred_bhv.to_csv('Yang_out_of_sample.csv')
ts_df.to_csv('Yang_test.csv')